In [159]:
%cd /home/druhe/rail1/src/cnf
%load_ext autoreload
%autoreload 2

import os
os.environ['DATAROOT'] = '/home/druhe/datasets'


/home/druhe/rail1/src/cnf
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/druhe/rail1/src/cnf/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [160]:
import torch
from pointnet2_ops import pointnet2_utils


In [161]:
xyz = torch.randn(1, 8, 3).cuda()
xyz = xyz.repeat(2, 1, 1)

In [162]:
x = pointnet2_utils.furthest_point_sample(xyz, 3)

In [163]:
num_samples = 3

In [164]:
import torch

def drop_from_tensor(tensor, index):
    B, N = tensor.shape[:2]
    range_tensor = torch.arange(N, device=tensor.device).unsqueeze(0).repeat(B, 1)
    mask = range_tensor != index.unsqueeze(1)
    dropped = tensor[mask].reshape(B, N - 1, 3)

    expanded_indices = index.unsqueeze(-1).unsqueeze(-1).expand(-1, -1, 3)
    values = torch.gather(tensor, 1, expanded_indices).squeeze(1)

    return dropped, values

In [178]:
ix = torch.zeros((len(xyz)), dtype=torch.int64, device=xyz.device)
# ix = torch.arange((len(xyz)), dtype=torch.int64, device=xyz.device)
indices = [ix]
B, N = xyz.shape[:2]
range_tensor = torch.arange(N, device=xyz.device).unsqueeze(0).repeat(B, 1)
mask = range_tensor != ix.unsqueeze(1)

print(mask)

for i in range(num_samples-1):
    points = torch.gather(xyz, 1, ix.unsqueeze(-1).unsqueeze(-1).expand(-1, -1, 3))
    d = torch.cdist(points, xyz).squeeze(1)

    d *= mask

    ix = torch.argmax(d, dim=1)

    indices.append(ix)
    mask = mask & (range_tensor != ix.unsqueeze(1))
    
#     xyz = torch.cat([xyz[:, :idx, :], xyz[:, idx+1:, :]], dim=1)
# point = xyz[:, start_idx: start_idx+1, :]
# d = torch.cdist(point, xyz)
# ix = torch.argmax(d, dim=2)
# indices.append(ix)
# point = torch.gather(xyz, 1, ix.unsqueeze(-1).expand(-1, -1, 3))
# d = torch.cdist(point, xyz)
# ix = torch.argmax(d, dim=2)
# indices.append(ix.unsqueeze(-1))

tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True,  True]],
       device='cuda:0')


In [166]:
indices

[tensor([0, 0], device='cuda:0'),
 tensor([4, 4], device='cuda:0'),
 tensor([7, 7], device='cuda:0')]

In [171]:
x_i = torch.stack(indices, dim=1)

In [172]:
x_i.shape, x.shape

(torch.Size([2, 3]), torch.Size([2, 3]))

In [181]:
npoint = 3

xyz = torch.randn(1, 3, 3)

device = xyz.device
B, N, C = xyz.shape
centroids = torch.zeros(B, npoint, dtype=torch.long).to(device)
distance = torch.ones(B, N).to(device) * 1e10
batch_indices = torch.arange(B, dtype=torch.long).to(device)

farthest = torch.zeros(B, dtype=torch.long).to(device)

for i in range(npoint):
    centroids[:, i] = farthest
    centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
    dist = torch.sum((xyz - centroid) ** 2, -1)
    print(dist)
    print(distance)
    print('\n')
    mask = dist < distance
    distance[mask] = dist[mask]
    farthest = torch.max(distance, -1)[1]


tensor([[0.0000, 1.5031, 5.0591]])
tensor([[1.0000e+10, 1.0000e+10, 1.0000e+10]])


tensor([[5.0591, 6.0484, 0.0000]])
tensor([[0.0000, 1.5031, 5.0591]])


tensor([[1.5031, 0.0000, 6.0484]])
tensor([[0.0000, 1.5031, 0.0000]])


